<img 
    style="position: absolute; 
           left: 60%; 
           top: 0; /* Added to ensure proper positioning */
           height: 900px; 
           width: 40%; /* Maintain the original width */
           object-fit: cover; /* Adjust if necessary */
           clip-path: inset(0px 50px 0px 50px round 10px);" 
    src= "analytical/buildingdamage.svg"
/>
</figure>


<h1 style="width: 60%; color: #EC6842; font-size: 55pt;">
    <Strong>
        Empirical analytical damage assessment 
    </Strong>
</h1>


<p style="text-align: justify; width: 60%; font-weight: normal;">
     This notebook provides an analytical assessment of a detached, two-story unreinforced masonry structure built in the 1960s and later demolished. The assessment is based on a foundation research report conducted prior to the building's demolition, which documented the structure’s measurements, characteristics, damage state, and the effects of subsidence.
</p>

## <strong> X | Imports</strong>

In [13]:
import analytical as ba
from analytical.assessment.tools.plots import *

import numpy as np

## <strong> 0 | Instantiate `HOUSE` object</strong>

### <strong> 0.1 | Structure dimensions and measurements</strong>

In [14]:
walls = {
    'Wall 1':{"x": np.array([0, 0, 0]), "y": np.array([0, 3.5, 7]), "z": np.array([0, -72, -152]), 'phi': np.array([1/200,1/200]), 'height': 5250, 'thickness': 27,'area': 34.25,'area': 34.25, 'opening': 4.86},
    'Wall 2':{"x": np.array([0, 4.5, 8.9]), "y": np.array([7, 7, 7]), "z": np.array([-152, -163, -188]),  'phi': np.array([1/33,1/50]), 'height': 5250, 'thickness': 27,'area': 37,'opening': 9.36},
    'Wall 3':{"x": np.array([8.9, 8.9]), "y": np.array([3.6, 7]), "z": np.array([-149, -188]), 'phi': np.array([0,0]), 'height': 5250, 'thickness': 27,'area': 24.35, 'opening': 4.98},
    'Wall 4':{"x": np.array([8.9, 10.8]), "y": np.array([3.6, 3.6]), "z": np.array([-149,-138]), 'phi': np.array([0,0]), 'height': 2850, 'thickness': 27,'area': 8.09, 'opening': 1.68},
    'Wall 5':{"x": np.array([10.8, 10.8]), "y": np.array([0, 3.6]), "z": np.array([-104, -138]), 'phi': np.array([0,0]), 'height': 2850, 'thickness': 27,'area': 9.15, 'opening': 1},
    'Wall 6':{"x": np.array([0, 5.2, 6.4, 8.9, 10.8]), "y": np.array([0, 0, 0, 0, 0]), "z": np.array([0, -42, -55, -75, -104]), 'phi': np.array([1/100,1/100]), 'height': 5000, 'thickness': 27, 'area': 47.58, 'opening': 4.42},
}

damage =  {'crack_1' : {'wall_id' : 'Wall 2','c_w': 4,'c_l': 890,},
           'crack_2' : {'wall_id' : 'Wall 1','c_w': 2,'c_l': 1670,},
           'crack_3' : {'wall_id' : 'Wall 2','c_w': 3,'c_l': 1868,},
           'crack_5' : {'wall_id' : 'Wall 2','c_w': 3,'c_l': 1760,},
           'crack_6' : {'wall_id' : 'Wall 2','c_w': 3,'c_l': 1200,},
           'crack_7' : {'wall_id' : 'Wall 2','c_w': 2,'c_l': 600,},
           'crack_9' : {'wall_id' : 'Wall 2','c_w': 2,'c_l': 994,},
           'crack_10': {'wall_id' : 'Wall 2','c_w': 2,'c_l': 980,}
           }

ijsselsteinseweg = ba.house(measurements = walls)
ijsselsteinseweg.state = damage

### <strong> 0.2 | Estimating the displacement surface & greenfield profile</strong>

In [15]:
ijsselsteinseweg.interpolate() # Interpolate subsidence shape
ijsselsteinseweg.fit_function(i_guess = 1, tolerance = 1e-2, step = 1) # Fit gaussian shapes to walls

params = ijsselsteinseweg.soil['house'].values()
app = subsurface(ijsselsteinseweg, *params)
app.run_server(port=8050, debug=False)

## <strong> 1 | Assessing the damage of the building</strong>

### <strong> 1.1 | Assesment through Empirical Methods </strong>

In [17]:
ijsselsteinseweg.SRI(tolerance= 0.01) #Compute the SRI parameters
report = ba.EM(ijsselsteinseweg.soil['sri'])

app = EM_plot(report)
app.run_server(port=8051, debug=False)

### <strong> 1.2 | Assesment through the LTSM </strong>

In [18]:
limit_line = -1
ba.LTSM(ijsselsteinseweg, limit_line, methods = ['greenfield','measurements'])

app = LTSM_plot(ijsselsteinseweg)    
app.run_server(port=8052, debug=True, dev_tools_ui=False)